In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pyspark.sql import SparkSession
import pyspark.sql as sparksql

spark = SparkSession.builder.appName('stroke').getOrCreate()
train = spark.read.csv('/Users/manshusharma/Desktop/stroke prediction/train_2v.csv', inferSchema=True,header=True)
test = spark.read.csv('/Users/manshusharma/Desktop/stroke prediction/test_2v.csv', inferSchema=True,header=True)

In [20]:
#what kind of data yaar
train.printSchema()
train.dtypes
test.printSchema()
test.dtypes

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)



[('id', 'int'),
 ('gender', 'string'),
 ('age', 'double'),
 ('hypertension', 'int'),
 ('heart_disease', 'int'),
 ('ever_married', 'string'),
 ('work_type', 'string'),
 ('Residence_type', 'string'),
 ('avg_glucose_level', 'double'),
 ('bmi', 'double'),
 ('smoking_status', 'string')]

In [21]:
#Few list
train.toPandas().head(21)



,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,None,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,None,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,None,0
5,32257,Female,47.0,0,0,Yes,Private,Urban,210.95,50.1,None,0
6,52800,Female,52.0,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
7,41413,Female,75.0,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
8,15266,Female,32.0,0,0,Yes,Private,Rural,77.67,32.3,smokes,0
9,28674,Female,74.0,1,0,Yes,Self-employed,Urban,205.84,54.6,never smoked,0


In [22]:
test.toPandas().head(5)



,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,36306,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
1,61829,Female,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked
2,14152,Female,14.0,0,0,No,children,Rural,95.16,21.2,None
3,12997,Male,28.0,0,0,No,Private,Urban,94.76,23.4,None
4,40801,Female,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked


In [23]:
train.groupBy('stroke').count().show()

+------+-----+
|stroke|count|
+------+-----+
|     1|  783|
|     0|42617|
+------+-----+



In [24]:
# fill in missing values for smoking status
# As this is categorical data, we will add one data type "No Info" for the missing one
train_f = train.na.fill('No Info', subset=['smoking_status'])
test_f = test.na.fill('No Info', subset=['smoking_status'])



In [25]:
test_f.toPandas().head(50)



,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,36306,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
1,61829,Female,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked
2,14152,Female,14.0,0,0,No,children,Rural,95.16,21.2,No Info
3,12997,Male,28.0,0,0,No,Private,Urban,94.76,23.4,No Info
4,40801,Female,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked
5,9348,Female,66.0,1,0,Yes,Private,Urban,219.98,32.2,never smoked
6,51550,Female,49.0,0,0,Yes,Self-employed,Rural,74.03,25.1,No Info
7,60512,Male,46.0,0,0,Yes,Govt_job,Urban,120.80,32.5,never smoked
8,31309,Female,75.0,0,0,Yes,Self-employed,Rural,78.71,28.0,never smoked
9,39199,Male,75.0,0,0,Yes,Self-employed,Urban,77.20,25.7,smokes


In [26]:
# fill in miss values for bmi 
# as this is numecial data , we will simple fill the missing values with mean
from pyspark.sql.functions import mean
mean = train_f.select(mean(train_f['bmi'])).collect()
mean_bmi = mean[0][0]
train_f = train_f.na.fill(mean_bmi,['bmi'])
test_f = test_f.na.fill(mean_bmi,['bmi'])

In [27]:
test_f.toPandas().head(50)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,36306,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
1,61829,Female,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked
2,14152,Female,14.0,0,0,No,children,Rural,95.16,21.2,No Info
3,12997,Male,28.0,0,0,No,Private,Urban,94.76,23.4,No Info
4,40801,Female,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked
5,9348,Female,66.0,1,0,Yes,Private,Urban,219.98,32.2,never smoked
6,51550,Female,49.0,0,0,Yes,Self-employed,Rural,74.03,25.1,No Info
7,60512,Male,46.0,0,0,Yes,Govt_job,Urban,120.80,32.5,never smoked
8,31309,Female,75.0,0,0,Yes,Self-employed,Rural,78.71,28.0,never smoked
9,39199,Male,75.0,0,0,Yes,Self-employed,Urban,77.20,25.7,smokes


In [28]:
# indexing all categorical columns in the dataset
from pyspark.ml.feature import StringIndexer
indexer1 = StringIndexer(inputCol="gender", outputCol="genderIndex")
indexer2 = StringIndexer(inputCol="ever_married", outputCol="ever_marriedIndex")
indexer3 = StringIndexer(inputCol="work_type", outputCol="work_typeIndex")
indexer4 = StringIndexer(inputCol="Residence_type", outputCol="Residence_typeIndex")
indexer5 = StringIndexer(inputCol="smoking_status", outputCol="smoking_statusIndex")

In [29]:
# Doing one hot encoding of indexed data
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["genderIndex","ever_marriedIndex","work_typeIndex","Residence_typeIndex","smoking_statusIndex"],
                                 outputCols=["genderVec","ever_marriedVec","work_typeVec","Residence_typeVec","smoking_statusVec"])

In [30]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['genderVec',
 'age',
 'hypertension',
 'heart_disease',
 'ever_marriedVec',
 'work_typeVec',
 'Residence_typeVec',
 'avg_glucose_level',
 'bmi',
 'smoking_statusVec'],outputCol='features')

In [31]:
from pyspark.ml.classification import DecisionTreeClassifier
dtc = DecisionTreeClassifier(labelCol='stroke',featuresCol='features')

In [32]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer1, indexer2, indexer3, indexer4, indexer5, encoder, assembler, dtc])

In [33]:
# splitting training and validation data
train_data,val_data = train_f.randomSplit([0.7,0.3])

# training model pipeline with data
model = pipeline.fit(train_data)

In [34]:
# making prediction on model with validation data
dtc_predictions = model.transform(val_data)

# Select example rows to display.
dtc_predictions.select("prediction","probability", "stroke", "features").show(11)

+----------+--------------------+------+--------------------+
|prediction|         probability|stroke|            features|
+----------+--------------------+------+--------------------+
|       0.0|[0.99180614345073...|     0|(16,[0,2,5,6,11,1...|
|       0.0|[0.99180614345073...|     0|(16,[0,2,6,10,11,...|
|       0.0|[0.99180614345073...|     0|(16,[0,2,5,6,11,1...|
|       0.0|[0.99180614345073...|     0|(16,[1,2,5,9,10,1...|
|       0.0|[0.99180614345073...|     0|(16,[1,2,8,11,12,...|
|       0.0|[0.99180614345073...|     0|(16,[1,2,8,11,12,...|
|       0.0|[0.99180614345073...|     0|(16,[0,2,6,10,11,...|
|       0.0|[0.92145369284876...|     0|(16,[1,2,5,9,11,1...|
|       0.0|[0.99180614345073...|     0|(16,[0,2,5,9,10,1...|
|       0.0|[0.99180614345073...|     0|(16,[0,2,5,6,10,1...|
|       0.0|[0.99180614345073...|     0|(16,[1,2,5,6,11,1...|
+----------+--------------------+------+--------------------+
only showing top 11 rows



In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="stroke", predictionCol="prediction", metricName="accuracy")
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
print('A Decision Tree algorithm had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))

A Decision Tree algorithm had an accuracy of: 98.15%


In [36]:
# now predicting the labels for test data
test_pred = model.transform(test_f)
test_selected = test_pred.select("id", "features", "prediction","probability")
test_selected.limit(100).toPandas()

,id,features,prediction,probability
0,36306,"(0.0, 1.0, 80.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0,...",0.0,"[0.9403444034440345, 0.05965559655596556]"
1,61829,"(1.0, 0.0, 74.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0,...",0.0,"[0.9214536928487691, 0.07854630715123095]"
2,14152,"(1.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",0.0,"[0.9918061434507398, 0.008193856549260223]"
3,12997,"(0.0, 1.0, 28.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...",0.0,"[0.9918061434507398, 0.008193856549260223]"
4,40801,"(1.0, 0.0, 63.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...",0.0,"[0.9918061434507398, 0.008193856549260223]"
5,9348,"(1.0, 0.0, 66.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0,...",0.0,"[0.9918061434507398, 0.008193856549260223]"
6,51550,"(1.0, 0.0, 49.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0,...",0.0,"[0.9918061434507398, 0.008193856549260223]"
7,60512,"(0.0, 1.0, 46.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...",0.0,"[0.9918061434507398, 0.008193856549260223]"
8,31309,"(1.0, 0.0, 75.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0,...",0.0,"[0.9403444034440345, 0.05965559655596556]"
9,39199,"(0.0, 1.0, 75.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0,...",0.0,"[0.9403444034440345, 0.05965559655596556]"
